In [4]:
# 🧩 Install if needed
!pip install gradio torch torchvision pillow

import gradio as gr
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from PIL import Image

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 30.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.4 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 12.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 65.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.3/95.3 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 78.2 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━

In [5]:
# 🧠 Define the ResNet9 model
class ResNet9(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(ResNet9, self).__init__()
        def conv_block(in_channels, out_channels, pool=False):
            layers = [nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1),
                      nn.BatchNorm2d(out_channels),
                      nn.ReLU(inplace=True)]
            if pool: layers.append(nn.MaxPool2d(2))
            return nn.Sequential(*layers)

        self.conv1 = conv_block(in_channels, 64)
        self.conv2 = conv_block(64, 128, pool=True)
        self.res1 = nn.Sequential(conv_block(128, 128), conv_block(128, 128))
        self.conv3 = conv_block(128, 256, pool=True)
        self.conv4 = conv_block(256, 512, pool=True)
        self.res2 = nn.Sequential(conv_block(512, 512), conv_block(512, 512))
        self.classifier = nn.Sequential(
            nn.MaxPool2d(4),
            nn.Flatten(),
            nn.Linear(512 * 4 * 4, 38)
        )

    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.res1(x) + x
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.res2(x) + x
        return self.classifier(x)


In [8]:
# 📥 Load model (update the path to your model file)
from torch.serialization import add_safe_globals

# 1. Register your custom class for safe loading
add_safe_globals({'ResNet9': ResNet9})

# 2. Load the full model object
model_path = "/kaggle/input/plant-disease-01/plant-disease-model-complete.pth"
model = torch.load(model_path, map_location='cpu', weights_only=False)
model.eval()


ResNet9(
  (conv1): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
  )
  (conv2): Sequential(
    (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=4, stride=4, padding=0, dilation=1, ceil_mode=False)
  )
  (res1): Sequential(
    (0): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=True)
    )
    (1): Sequential(
      (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace=Tr

In [9]:
# 🔁 Define image transform
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor()
])

# 🏷 Class names dictionary
class_names = {
    0: 'Tomato___Late_blight', 1: 'Tomato___healthy', 2: 'Grape___healthy', 3: 'Orange___Haunglongbing_(Citrus_greening)',
    4: 'Soybean___healthy', 5: 'Squash___Powdery_mildew', 6: 'Potato___healthy', 7: 'Corn_(maize)___Northern_Leaf_Blight',
    8: 'Tomato___Early_blight', 9: 'Tomato___Septoria_leaf_spot', 10: 'Corn_(maize)___Cercospora_leaf_spot_Gray_leaf_spot',
    11: 'Strawberry___Leaf_scorch', 12: 'Peach___healthy', 13: 'Apple___Apple_scab', 14: 'Tomato___Tomato_Yellow_Leaf_Curl_Virus',
    15: 'Tomato___Bacterial_spot', 16: 'Apple___Black_rot', 17: 'Blueberry___healthy', 18: 'Cherry_(including_sour)___Powdery_mildew',
    19: 'Peach___Bacterial_spot', 20: 'Apple___Cedar_apple_rust', 21: 'Tomato___Target_Spot', 22: 'Pepper,_bell___healthy',
    23: 'Grape___Leaf_blight_(Isariopsis_Leaf_Spot)', 24: 'Potato___Late_blight', 25: 'Tomato___Tomato_mosaic_virus',
    26: 'Strawberry___healthy', 27: 'Apple___healthy', 28: 'Grape___Black_rot', 29: 'Potato___Early_blight',
    30: 'Cherry_(including_sour)___healthy', 31: 'Corn_(maize)___Common_rust_', 32: 'Grape___Esca_(Black_Measles)',
    33: 'Raspberry___healthy', 34: 'Tomato___Leaf_Mold', 35: 'Tomato___Spider_mites_Two-spotted_spider_mite',
    36: 'Pepper,_bell___Bacterial_spot', 37: 'Corn_(maize)___healthy'
}



In [10]:
# 🔍 Prediction function
def predict(img):
    img = img.convert('RGB')
    img_tensor = transform(img).unsqueeze(0)
    with torch.no_grad():
        output = model(img_tensor)
        _, predicted = torch.max(output, 1)
        label = predicted.item()
        return class_names[label]

# 🚀 Create Gradio Interface
gr.Interface(
    fn=predict,
    inputs=gr.Image(type='pil'),
    outputs=gr.Label(),
    title="🌿 Plant Disease Detection",
    description="Upload a plant leaf image to detect the disease using a ResNet9 model trained on 38 classes."
).launch()


* Running on local URL:  http://127.0.0.1:7860
It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

* Running on public URL: https://82a74d727cac28c573.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
